In [52]:
import numpy as np
import time
import soundfile as sf
import math

In [55]:
tube_length = 80
fs = 44100
embochure = 20
inputGain = 0.1
nbr_de_sec = 0.1
duration = int(fs*nbr_de_sec)
voicedGain = 1
outputGain = 1

In [56]:
def noise():
    return (np.random.uniform() - 0.5) * 2

def normalize(array):
    min = np.min(array)
    max = np.max(array)
    normalized_array = 2*(array-min)/(max-min) - 1
    return normalized_array


In [68]:
def hole1(i,x):
    return x

def hole2(i,x):
    return x

In [67]:
# Valimaki 1993

# Variables 変数
# Preallocate output array for efficiency
output = np.zeros(duration)  
delayEmb = np.zeros(duration)

delayTube = np.zeros(duration)
delayOutHoles = np.empty(duration)

delayHole1 = np.empty(duration)
delayHole1Distance = np.empty(duration)
delayHole1Decorr1 = np.empty(duration)
delayHole1Decorr2 = np.empty(duration)

delayHole2 = np.empty(duration)
delayHole2Distance = np.empty(duration)
delayHole2Decorr1 = np.empty(duration)
delayHole2Decorr2 = np.empty(duration)

output1 = 0
output2 = 0
feedback1 = 0
feedback2 = 0
feedback3 = 0

Dfdbk = 0.99
fdbk_gain = 0.98

hole1Distance = 40
hole2Distance = 60

hole1Corr = 0
historyBoundaryLossFilter = 0
historyDCkiller = 0
branchDCkiller = 0
historyReflectionFilter = 0
historyHole1Filter = 0
historyHole2Filter = 0
delaydc = 0

start = 0
end = 0

coeff = 0.3333

start = time.time()
for i in range(duration):
    instantSample = noise() * inputGain

    # Voiced Gain feedback1+feedback2
    if  i != 0:
        instantSample = instantSample + ((feedback1+feedback2) * voicedGain)

    # Delay-line Embouchure
    delayEmb[i] = instantSample
    if i > embochure:
        instantSample = delayEmb[i-embochure]

    # Nonlinearity
    instantSample -= (instantSample ** 3)

    # DC Killer
    instantSample_dckiller = instantSample
    DCKiller = instantSample_dckiller * 0.5 + delaydc * (-0.5)
    delaydc = DCKiller
    instantSample = instantSample_dckiller - DCKiller

    if i != 0:
        instantSample += feedback2

    # Boundary Loss Filter
    instantSample_blf = instantSample
    BLF = instantSample_blf * 0.5 + historyBoundaryLossFilter * 0.5
    historyBoundaryLossFilter = BLF
    instantSample = BLF

    # Tube Length delay
    delayTube[i] = instantSample
    if i > tube_length:
        instantSample = delayTube[i - tube_length]
    instantSample_to_output = instantSample
    instantSample_to_reflection = instantSample * 0.3333
    instantSample_to_holes = instantSample * 0.3333
    
    outHole1 = hole1(i, instantSample_to_holes)
    outHole2 = hole2(i, instantSample_to_holes)
    outputHoles = outHole1 + outHole2

    # Reflection Filter
    reflectionFilter = (instantSample_to_reflection * 0.5 + historyReflectionFilter * 0.5) * 0.897
    historyReflectionFilter = reflectionFilter

    output[i] = (0.333 * instantSample_to_output + outputHoles) * outputGain

    delayOutHoles[i] = outputHoles + reflectionFilter
    if i > tube_length:
        newSample = delayOutHoles[i - tube_length]
    else:
        newSample = 0
    
    feedback1 = DCKiller * Dfdbk
    feedback2 = newSample * fdbk_gain

normalized_output = normalize(output)
print(normalized_output)
end = time.time()

[nan nan nan ... nan nan nan]


/var/folders/15/s_s_6k8d24z5bpljjglt6m_00000gn/T/ipykernel_18799/130747855.py:7: RuntimeWarning: invalid value encountered in divide
  normalized_array = 2*(array-min)/(max-min) - 1


In [ ]:
# write output to file for listening
sf.write('output_Valimaki92.wav', normalized_output, fs)

elapsed_time = end - start
print(elapsed_time)

0.8612298965454102
